In [1]:
import cv2
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import glob
# For our model
import torchvision.models as convels
import matplotlib.pyplot as plt
import numpy as np
# For utilities
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from torch.nn.functional import normalize
from sys import platform
from itertools import combinations
import torchvision.transforms as T
from time import time

In [2]:
def load(folder):
    files = glob.glob(folder)
    data =[]
    for f in files:
        image = cv2.imread(f)
        data.append(image)
    return data


def group(data, album_length):
    #group into chunks of three because of three sets of images in LAB color space
    for i in range (0, album_length, 3):
        yield image_data[i:i+3]


class SplitLab(object):
    """Splits tensor LAB image to L and ab channels."""
    def __call__(self, image):
        L  = image[:1,:,:]
        ab = image[1:,:,:]
        return (L, ab)
    

class ImageDataset(Dataset):
    """
       Custom dataset for LAB image preprocessing for colorization
    """
    def __init__(self, images):
       """
       images: array of lab images [idx, h, w, channels],
       where for channels 0 = L, 1 = a, 2 = b.
       """
       self.images = images
       self.composed = T.Compose([T.ToPILImage(),T.ToTensor(),SplitLab()])

        
    def __len__(self):
        return len(self.images)
      
    
    def __getitem__(self, idx):
        """
        returns L, ab mean, and ab for visualization
        """
        image = self.images[idx, :, :, :]
        L, ab = self.composed(image)

        ab_mean = torch.mean(ab, dim=[1, 2])

        sample = {'L': L, 'ab': ab, 'ab_mean': ab_mean}
        
        return sample

In [3]:
home_dir = os.getcwd() 
#change this parameter depending on which album you want
target_album = 'LAB_TEST_FACES'
batch_size = 32

# load in data and split into test and train sets
image_data = np.asarray(load(home_dir + '/' + target_album + '/' + '*.jpg'))
train_images, test_images = train_test_split(image_data, test_size = 0.1, random_state=42)

# dataset and dataloader to prepare images for training
train_dataset = ImageDataset(train_images)
test_dataset = ImageDataset(test_images)

train_loader = DataLoader(dataset = train_dataset, batch_size = batch_size, shuffle=True)
test_loader = DataLoader(dataset = test_dataset, batch_size = batch_size, shuffle=True)

In [4]:
print(len(image_data))

2250


In [5]:
# select GPU / CPU
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
print('Device:', device)

Device: cuda:0


In [6]:
class ChrominanceReg(nn.Module):
    def __init__(self):
        super(ChrominanceReg, self).__init__()

        #128x128
        self.conv1 = nn.Sequential(
            nn.Conv2d(1, 1, kernel_size=3, stride=2), # 128
            nn.ReLU()
        )
        #64x64
        self.conv2 = nn.Sequential(
            nn.Conv2d(1, 1, kernel_size=3, stride=2), # 64
            nn.ReLU()
        )
        #32x32
        self.conv3 = nn.Sequential(
            nn.Conv2d(1, 1, kernel_size=3, stride=2), # 32
            nn.ReLU()
        )
        #16x16
        self.conv4 = nn.Sequential(
            nn.Conv2d(1, 1, kernel_size=3, stride=2), # 16
            nn.ReLU()
        )
        #8x8
        self.conv5 = nn.Sequential(
            nn.Conv2d(1, 1, kernel_size=3, stride=2), # 8
            nn.ReLU()
        )
        #4x4
        self.conv6 = nn.Sequential(
            nn.Conv2d(1, 1, kernel_size=3, stride=2), # 4
            nn.ReLU()
        )
        #2x2
        self.conv7 = nn.Sequential(
            nn.Conv2d(1, 1, kernel_size=2, stride=2), # 2
            nn.ReLU()
        )
        
        self.out = nn.Sequential(                     #1
            nn.Linear(1, 2)
        )

    def forward(self, x):
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.conv5(x)
        x = self.conv6(x)
        x = self.conv7(x)
        output = self.out(x)
        return output


regressor = ChrominanceReg().to(device)
print(regressor)


ChrominanceReg(
  (conv1): Sequential(
    (0): Conv2d(1, 1, kernel_size=(3, 3), stride=(2, 2))
    (1): ReLU()
  )
  (conv2): Sequential(
    (0): Conv2d(1, 1, kernel_size=(3, 3), stride=(2, 2))
    (1): ReLU()
  )
  (conv3): Sequential(
    (0): Conv2d(1, 1, kernel_size=(3, 3), stride=(2, 2))
    (1): ReLU()
  )
  (conv4): Sequential(
    (0): Conv2d(1, 1, kernel_size=(3, 3), stride=(2, 2))
    (1): ReLU()
  )
  (conv5): Sequential(
    (0): Conv2d(1, 1, kernel_size=(3, 3), stride=(2, 2))
    (1): ReLU()
  )
  (conv6): Sequential(
    (0): Conv2d(1, 1, kernel_size=(3, 3), stride=(2, 2))
    (1): ReLU()
  )
  (conv7): Sequential(
    (0): Conv2d(1, 1, kernel_size=(2, 2), stride=(2, 2))
    (1): ReLU()
  )
  (out): Sequential(
    (0): Linear(in_features=1, out_features=2, bias=True)
  )
)


In [7]:
epochs = 50
lr = 0.01
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(regressor.parameters(), lr)

regressor.train()

time0 = time()

for epoch in range(epochs):  # loop over the dataset multiple times

    running_loss = 0.0
    for data in train_loader:
        # get data
        L = data['L']
        ab_mean = data['ab_mean']

        # send to device
        L = L.to(device)
        ab_mean = ab_mean.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        outputs = regressor(L).squeeze()
        loss = loss_fn(outputs, ab_mean)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    else:
        print("Epoch {} - Training loss: {}".format(epoch, running_loss/len(train_loader)))

    if epoch % 4 == 0:
        y_pred = []
        y_true = []

        with torch.no_grad():
            regressor.eval()
            for data in test_loader:
                val_outputs = regressor(data['L'].to(device))
                val_loss = loss_fn(val_outputs.squeeze(), data['ab_mean'].to(device))

        print("\nValidation Loss = ", (val_loss))
print("\nTraining Finished. \nTraining Time (in minutes) =",(time()-time0)/60)



Epoch 0 - Training loss: 0.08002290112199262

Validation Loss =  tensor(0.0062, device='cuda:0')


c:\Users\hkirkland\.conda\envs\ML\lib\site-packages\torch\nn\modules\loss.py:520: UserWarning: Using a target size (torch.Size([1, 2])) that is different to the input size (torch.Size([2])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 1 - Training loss: 0.009181656711007236
Epoch 2 - Training loss: 0.009003997140098363
Epoch 3 - Training loss: 0.009073038643691689
Epoch 4 - Training loss: 0.009011852380353957

Validation Loss =  tensor(0.0194, device='cuda:0')
Epoch 5 - Training loss: 0.009069425439520273
Epoch 6 - Training loss: 0.009054453672433738
Epoch 7 - Training loss: 0.009016628820972983
Epoch 8 - Training loss: 0.009059438918484375

Validation Loss =  tensor(0.0194, device='cuda:0')
Epoch 9 - Training loss: 0.00904452920804033
Epoch 10 - Training loss: 0.009023499238537624
Epoch 11 - Training loss: 0.00900546721095452
Epoch 12 - Training loss: 0.009097065667447168

Validation Loss =  tensor(0.0025, device='cuda:0')
Epoch 13 - Training loss: 0.009261356884962879
Epoch 14 - Training loss: 0.008988868874439504
Epoch 15 - Training loss: 0.009243179112672806
Epoch 16 - Training loss: 0.009056861184944864

Validation Loss =  tensor(0.0061, device='cuda:0')
Epoch 17 - Training loss: 0.009162025795376394
Epoc

In [12]:
pred = []
true = []

with torch.no_grad():
    regressor.eval()
    for data in test_loader:
        true.extend(data['ab_mean'].numpy())
        outputs = regressor(data['L'].to(device))
        pred.extend(outputs.cpu().squeeze().numpy())

In [15]:
pred_np = np.stack(pred)
true_np = np.stack(true)

ValueError: all input arrays must have the same shape